In [1]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
from qiskit import ClassicalRegister
from LogicalQ.Benchmarks import mirror_benchmarking
from LogicalQ.NoiseModel import construct_noise_model
from LogicalQ.Logical import LogicalCircuit, LogicalStatevector
from LogicalQ.Experiments import execute_circuits
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_model_Quantinuum_H2_1

In [2]:
n_qubits = 1
circuit_length = 2

qec_params = {
    0: [5, 10],
    # 1: [5, 10]
}

In [7]:
mb_circ = mirror_benchmarking(
    n_qubits=n_qubits,
    circuit_length=circuit_length
)

def run_mb(
    qec_cycle_indices=None,
    noise_model=None
):

    # 2) lift into a LogicalCircuit (handles regs for you)
    logical_circ = LogicalCircuit.from_physical_circuit(
        mb_circ, **steane_code, name="mb_with_qec"
    )

    # 3) insert QEC cycles
    logical_circ.insert_qec_cycles(
        logical_qubit_indices=list(qec_cycle_indices.keys()),
        qec_cycle_indices=qec_cycle_indices,
        clear_existing_qec=False
    )

    # 4) measure all
    logical_circ.measure_all()

    print(logical_circ.count_ops())

    # 6) execute
    job = execute_circuits(
        logical_circ,
        backend="aer_simulator",
        hardware_model=hardware_model_Quantinuum_H2_1, coupling_map=None, noise_model=noise_model,
        method="statevector", shots=1E4
    )

    print(job)

    # 7) pull out counts & rebuild logical statevector
    raw_counts = job[0].get_counts()
    lsv = LogicalStatevector.from_counts(
        raw_counts,
        n_logical_qubits=1,
        **steane_code
    )

    print(lsv.logical_decomposition)

    # 8) return logical infidelity = 1 − |⟨0_L|ψ⟩|²
    return 1 - (lsv.logical_decomposition[0])**2

# 1. Ideal

In [8]:
infidelities = []
infidelity = run_mb(
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

OrderedDict({'measure': 7, 'box': 4, 'x': 1})
[Result(backend_name='aer_simulator_statevector', backend_version='0.17.1', job_id='b01ac4c1-d815-4fc9-b2a0-57c95fa83232', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x72000000': 1254, '0x6c000000': 1224, '0x1e000000': 1173, '0xc6000000': 1297, '0x0': 1239, '0xaa000000': 1273, '0xd8000000': 1277, '0xb4000000': 1263}), header={'creg_sizes': [['cenc_verif0', 1], ['ccurr_syndrome0', 3], ['cprev_syndrome0', 6], ['cflagged_syndrome_diff0', 6], ['cunflagged_syndrome_diff0', 6], ['cpauli_frame0', 2], ['clogical_op_meas0', 1], ['cfinal_meas0', 7], ['coutput', 1]], 'global_phase': 4.712388980384664, 'memory_slots': 33, 'n_qubits': 14, 'name': 'mb_with_qec', 'qreg_sizes': [['qlog0', 7], ['qanc0', 3], ['qlogical_op0', 2], ['qsetter', 2]], 'metadata': {}}, status=DONE, seed_simulator=1535974896, metadata={'time_taken': 4.181424885, 'num_bind_params': 1, 'parallel_state_update': 1,

# 2. Depolarizing (1q) on Clifford gates

In [ ]:
# Apply only single-qubit depolarizing errors to Clifford gates
noise2 = construct_noise_model(
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5*1E2
)

infidelity = run_mb(
    qec_cycle_indices=qec_params,
    noise_model=noise2
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 3. Depolarizing (1q) on all gates

In [ ]:
noise3 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=(1-0.54) * 1.89E-5*1E2
)
infidelity = run_mb(
    noise_model=noise3, 
    shots=shots, 
    n_logical_qubits=n_logical, 
    label=label, 
    stabilizer_tableau=stabilizer_tableau, 
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 4. Depolarizing (1q) + Depolarizing (2q) on all gates

In [ ]:
noise4 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=ERR_1Q,
    depolarizing_error_2q=ERR_2Q
)
infidelity = run_mb(
    noise_model=noise4, 
    shots=shots, 
    n_logical_qubits=n_logical, 
    label=label, 
    stabilizer_tableau=stabilizer_tableau, 
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 5. + Amplitude damping 1q

In [ ]:
noise5 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=ERR_1Q,
    depolarizing_error_2q=ERR_2Q,
    amplitude_damping_error_1q=AMP_1Q
)
infidelity = run_mb(
    noise_model=noise5, 
    shots=shots, 
    n_logical_qubits=n_logical, 
    label=label, 
    stabilizer_tableau=stabilizer_tableau, 
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# 6. + Readout errors

In [ ]:
noise6 = construct_noise_model(
    basis_gates=basis_gates_all,
    n_qubits=n_qubits,
    depolarizing_error_1q=ERR_1Q,
    depolarizing_error_2q=ERR_2Q,
    **{"readout_error_0|1": RD01, "readout_error_1|0": RD10},
    amplitude_damping_error_1q=AMP_1Q
)
infidelity = run_mb(
    noise_model=noise6, 
    shots=shots, 
    n_logical_qubits=n_logical, 
    label=label, 
    stabilizer_tableau=stabilizer_tableau, 
    qec_cycle_indices=qec_params
)

infidelities.append(infidelity)
print(f"Yes QEC: {infidelities}")

# Summary Plot